In [2]:
import pandas as pd
import numpy as np
import random
from useful_functions import generate_fake_data,bag_of_words_similarity,get_clusters,ratcliff_similarity
import matplotlib.pylab as plt

# Generate data

In [3]:
number_code = 600
number_provider = 1200

provider_column = 'provider'
code_column= 'code'
treatment_column = 'treatment'
threshold = 0.23

In [4]:
def generate_fake_data_mofified(number_code,number_provider,provider_column,code_column):
    
    # Define traitement , specialy and providers code
    speciality_code = []
    for i in range(number_code):
        speciality_code.append('C'+str(i))
        
    provider_code = []
    for i in range(number_provider):
        provider_code.append('P'+str(i))
    # construct fake data : unbalanced
    data_base = []
    for provider in provider_code:
        
        c1 = random.randint(0,number_code)
        list_of_treatments = speciality_code[max(c1-100,0):min(c1+100,number_code)]
        
        number_traitements = random.randint(1,500)
        
        for treatment in range(number_traitements):
            code_tr = random.choice(list_of_treatments)
            data_base.append({'provider':provider,'code':code_tr})
            
    return pd.DataFrame(data_base)

In [5]:
data = generate_fake_data_mofified(number_code,number_provider,provider_column,code_column)

In [6]:
data.sample(10)

,code,provider
199394,C21,P817
80342,C112,P326
175709,C216,P718
273931,C453,P1119
23422,C92,P96
258333,C479,P1054
128258,C450,P525
122735,C243,P502
182478,C126,P750
284586,C500,P1165


<font color="red"> KEEP THIS FORMAT FOR LATER : DO NOT EXECUTE THIS PART OF CODE !!</font>

In [ ]:
from requests import ConnectionError
import pymongo
from pymongo import MongoClient
import optparse

In [ ]:
def create_mongo_db(drop_indicateur,data,provider_column,code_columns):
    
    client = MongoClient()
    db = client.peer_grouping
    
    if drop_indicateur:
        # drop the db
        db.enrichments_collection.drop()
    else:
        pass
    collection = db.enrichments_collection
    
    # if the db is not dropped and is not null , exist
    if db.enrichments_collection.count() >0:
        collection = db.enrichments_collection
        
    else:
        for provider, inter_df in data.groupby(provider_column):
            dff = pd.DataFrame(inter_df.reset_index()).copy()
            for code , inter_df_2 in dff.groupby(code_columns):
                temporary = pd.DataFrame(inter_df_2.reset_index()).copy()

                number_treatment = {'provider_id':provider,code:len(temporary)}
                collection.insert_one(number_treatment)
                
    return pd.DataFrame(list(collection.find()))

In [ ]:
# You need to open the mongo connection before executing the code
grouping_traitment = create_mongo_db(False,data,provider_column,code_columns)

In [ ]:
len(grouping_traitment)

In [ ]:
grouping_traitment.drop('_id', axis=1, inplace=True)

In [ ]:
grouping_traitment.fillna(0,inplace=True)

In [ ]:
formated_data = grouping_traitment.groupby('provider_id').sum().reset_index()

In [ ]:
len(formated_data)

In [ ]:
formated_data.head(2)

# A. Formating dataframe as text

## Meth 1 : Group the codes for each practician

In [7]:
traitement = []
medecin = []
for provider, inter_df in data.groupby(provider_column):
    dff = pd.DataFrame(inter_df.reset_index()).copy()
    s = ""
    for x in list(dff[code_column]):
        s+=' '+str(x)
    traitement.append(s)
    medecin.append(provider)

In [8]:
# for each doctor , the list of treatment is a text
traitement[0][:100]

' C593 C526 C532 C500 C577 C596 C524 C558 C556 C523 C528 C594 C549 C536 C597 C541 C588 C581 C506 C529'

In [10]:
medecin[:20]

['P0',
 'P1',
 'P10',
 'P100',
 'P1000',
 'P1001',
 'P1002',
 'P1003',
 'P1004',
 'P1005',
 'P1006',
 'P1007',
 'P1008',
 'P1009',
 'P101',
 'P1010',
 'P1011',
 'P1012',
 'P1013',
 'P1014']

## Meth 2 : Group the practicians for each treatment

In [11]:
providers_listing = []
code_listing = []
for code, inter_df in data.groupby(code_column):
    dff = pd.DataFrame(inter_df.reset_index()).copy()
    s = ""
    for x in list(dff[provider_column]):
        s+=' '+str(x)
    providers_listing.append(s)
    code_listing.append(code)

In [13]:
providers_listing[0]

' P4 P10 P10 P19 P28 P28 P30 P30 P30 P38 P38 P38 P43 P43 P50 P50 P50 P50 P73 P81 P81 P93 P93 P96 P96 P96 P100 P102 P106 P106 P106 P111 P127 P127 P127 P127 P130 P135 P135 P135 P135 P157 P157 P157 P157 P157 P172 P174 P174 P176 P176 P180 P180 P200 P221 P221 P222 P233 P233 P233 P236 P236 P240 P240 P240 P240 P240 P240 P244 P244 P254 P254 P256 P256 P261 P261 P261 P270 P270 P293 P293 P293 P306 P308 P308 P308 P308 P309 P335 P335 P335 P335 P335 P341 P348 P348 P348 P351 P351 P353 P355 P355 P355 P365 P365 P365 P378 P378 P379 P393 P393 P404 P404 P407 P418 P443 P443 P443 P443 P445 P445 P448 P457 P457 P457 P457 P457 P457 P466 P466 P466 P466 P470 P480 P480 P481 P481 P483 P483 P495 P495 P516 P516 P516 P516 P516 P516 P527 P527 P539 P539 P549 P549 P549 P549 P560 P585 P585 P587 P587 P591 P591 P591 P592 P597 P597 P609 P617 P617 P619 P619 P619 P619 P619 P626 P630 P630 P630 P631 P631 P631 P631 P631 P632 P632 P634 P634 P634 P634 P658 P660 P670 P678 P678 P679 P679 P679 P686 P687 P695 P695 P697 P700 P701 P705 

In [14]:
code_listing[0]

'C0'

# Peer grouping

## Bag of words similarity

### Applied to meth 1

In [15]:
similarity_matrix = bag_of_words_similarity(traitement)

In [16]:
dic_of_clusters = get_clusters(similarity_matrix,traitement,medecin,threshold)

In [17]:
print 'total number of clusters : {}'.format(len(dic_of_clusters))
for clu in range(len(dic_of_clusters)):
    print 'size of the {} cluster : {}'.format(clu,len(dic_of_clusters[clu]['cluster']))

total number of clusters : 4
size of the 0 cluster : 617
size of the 1 cluster : 573
size of the 2 cluster : 3
size of the 3 cluster : 2


### Applied to meth 2

In [18]:
similarity_matrix_meth2 = bag_of_words_similarity(providers_listing)

In [19]:
dic_of_clusters_meth2 = get_clusters(similarity_matrix_meth2,providers_listing,code_listing,threshold)

In [20]:
print 'total number of clusters : {}'.format(len(dic_of_clusters_meth2))
for clu in range(len(dic_of_clusters_meth2)):
    print 'size of the {} cluster : {}'.format(clu,len(dic_of_clusters_meth2[clu]['cluster']))

total number of clusters : 3
size of the 0 cluster : 303
size of the 1 cluster : 286
size of the 2 cluster : 11


## ratcliff_clustering

<font color="red"> NOT USEFULL : GIVE THE SAME RESULTS AND TAKE A LOT OF TIME TO BE EXECUTED !!!!</font>

### Applied to meth 1

In [ ]:
racliff_meth_1 = ratcliff_similarity(traitement)

In [ ]:
dic_of_clusters_racliff_meth1 = get_clusters(racliff_meth_1,traitement,medecin,threshold)

In [ ]:
print 'total number of clusters : {}'.format(len(dic_of_clusters_racliff_meth1))
for clu in range(len(dic_of_clusters_racliff_meth1)):
    print 'size of the {} cluster : {}'.format(clu,len(dic_of_clusters_racliff_meth1[clu]['cluster']))

### Applied to meth 2

In [ ]:
racliff_meth_2 = ratcliff_similarity(traitement)
dic_of_clusters_racliff_meth2 = get_clusters(racliff_meth_2,traitement,medecin,threshold)

In [ ]:
print 'total number of clusters : {}'.format(len(dic_of_clusters_racliff_meth1))
for clu in range(len(dic_of_clusters_racliff_meth1)):
    print 'size of the {} cluster : {}'.format(clu,len(dic_of_clusters_racliff_meth1[clu]['cluster']))

# B. Formating dataframe as distribution

In [21]:
all_used_codes = pd.DataFrame(list(set(data.code)),columns=['code'])

In [22]:
all_providers = []
all_distrib = []
for provider, inter_df in data.groupby(provider_column):
    dff = pd.DataFrame(inter_df.reset_index(drop = True)).copy()
    code_provi = dff.groupby('code').count().reset_index()
    all_codes = pd.merge(all_used_codes,code_provi, on ='code',how='left').fillna(0)
    number_treat_prov = sum(code_provi.provider)
    distrub_prov = list(all_codes['provider']/number_treat_prov)
    all_distrib.append(distrub_prov)
    all_providers.append(provider)

## Cosine distance

In [9]:
from sklearn.metrics.pairwise import pairwise_distances
similarity_matrix_prob = pairwise_distances(all_distrib, metric='cosine')
dicto_cluster = get_clusters(similarity_matrix_prob,all_distrib,all_providers,0)

In [10]:
print 'total number of clusters : {}'.format(len(dicto_cluster))
for clu in range(len(dicto_cluster)):
    print 'size of the {} cluster : {}'.format(clu,len(dicto_cluster[clu]['cluster']))

total number of clusters : 2
size of the 0 cluster : 668
size of the 1 cluster : 532


## Jensen–Shannon divergence

In [35]:
def jsdiv(P, Q):
    
    """Compute the Jensen-Shannon divergence between two probability distributions.

    Input
    -----
    P, Q : array-like
        Probability distributions of equal length that sum to 1
    """

    def _kldiv(A, B):
        return np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)])

    P = np.array(P)
    Q = np.array(Q)

    M = 0.5 * (P + Q)
    return 0.5 * (_kldiv(P, M) +_kldiv(Q, M))

In [36]:
nb_pres = len(all_distrib)
similarity_matrix = np.zeros((nb_pres, nb_pres))

# Fill the similarity matrix
for i in range(nb_pres):
    for j in range(nb_pres):
        if i < j:
            similarity = round(jsdiv(all_distrib[i], all_distrib[j]), 2)
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

In [37]:
dicto_cluster_jen = get_clusters(similarity_matrix,all_distrib,all_providers,0)

In [38]:
print 'total number of clusters : {}'.format(len(dicto_cluster_jen))
for clu in range(len(dicto_cluster_jen)):
    print 'size of the {} cluster : {}'.format(clu,len(dicto_cluster_jen[clu]['cluster']))

total number of clusters : 2
size of the 0 cluster : 652
size of the 1 cluster : 548


## Bhattacharyya distance

In [41]:
def bha_dist(P,Q):
    resul = []
    for i in range(len(P)):
        resul.append(np.sqrt(P[i]*Q[i]))
    return(-np.log2(sum(resul)))

In [42]:
nb_pres = len(all_distrib)
similarity_matrix = np.zeros((nb_pres, nb_pres))

# Fill the similarity matrix
for i in range(nb_pres):
    for j in range(nb_pres):
        if i < j:
            similarity = round(bha_dist(all_distrib[i], all_distrib[j]), 2)
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

In [43]:
dicto_cluster_bhatta = get_clusters(similarity_matrix,all_distrib,all_providers,0)

In [44]:
print 'total number of clusters : {}'.format(len(dicto_cluster_bhatta))
for clu in range(len(dicto_cluster_bhatta)):
    print 'size of the {} cluster : {}'.format(clu,len(dicto_cluster_bhatta[clu]['cluster']))

total number of clusters : 1
size of the 0 cluster : 1200


### Mahalanobis (particular case)

In [45]:
similarity_matrix_prob = pairwise_distances(all_distrib, metric='mahalanobis')
dicto_cluster = get_clusters(similarity_matrix_prob,all_distrib,all_providers,0)

print 'total number of clusters : {}'.format(len(dicto_cluster))
for clu in range(len(dicto_cluster)):
    print 'size of the {} cluster : {}'.format(clu,len(dicto_cluster[clu]['cluster']))

total number of clusters : 2
size of the 0 cluster : 640
size of the 1 cluster : 560
